<a href="https://colab.research.google.com/github/Shivanshu04/Case_Based_code/blob/main/correction_method_for_shifting_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import numpy as np

def transform_dataframe(df):
    # For each row, find the first non-NaN value from columns 3 onwards and replace NaN in the first two columns
    for index, row in df.iterrows():
        non_na_values = row[2:].dropna().values

        if pd.isna(row.iloc[0]) and len(non_na_values) > 0:
            df.at[index, df.columns[0]] = non_na_values[0]
            non_na_values = np.delete(non_na_values, 0)

        if pd.isna(row.iloc[1]) and len(non_na_values) > 0:
            df.at[index, df.columns[1]] = non_na_values[0]
            non_na_values = np.delete(non_na_values, 0)

        # Update the remaining columns with the leftover values from non_na_values
        for col_index, col in enumerate(df.columns[2:], start=2):
            if col_index - 2 < len(non_na_values):
                df.at[index, col] = non_na_values[col_index - 2]
            else:
                df.at[index, col] = np.nan

    return df

def process_file(file_path, output_folder):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Apply the transformation logic
    df = transform_dataframe(df)

    # Save the modified DataFrame to a new CSV file
    output_path = os.path.join(output_folder, os.path.basename(file_path))
    df.to_csv(output_path, index=False)

# List of file paths to process
file_paths = ["", "/path/to/file2.csv", ...]

# Output folder to save modified files
output_folder = "/path/to/output/folder"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each file
for file_path in file_paths:
    process_file(file_path, output_folder)


In [ ]:
# 1. Import necessary libraries
import tabula
import pandas as pd

# 2. Function to consolidate data based on primary columns
def consolidate_data(tables):
    # Define the primary columns
    primary_columns = ["Signatory ID", "Name", "CIN", "Company Name", "Defaulting Year"]

    # Create an empty DataFrame with the primary columns
    consolidated_df = pd.DataFrame(columns=primary_columns)

    # Iterate over each table and extract rows that contain the primary columns
    for table in tables:
        for _, row in table.iterrows():
            # Check if the primary columns are present in the row
            if any(col in row.values for col in primary_columns):
                # Extract relevant data and append to the consolidated DataFrame
                row_data = {col: row[col] if col in row else None for col in primary_columns}
                consolidated_df = consolidated_df.append(row_data, ignore_index=True)

    return consolidated_df

# 3. Function to extract tables from PDF and consolidate the data
def convert_pdf_to_excel_consolidated(pdf_path, output_path):
    # Extract tables from the PDF
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)

    # Consolidate the extracted data
    consolidated_table = consolidate_data(tables)

    # Write the consolidated table to Excel
    consolidated_table.to_excel(output_path, sheet_name="Consolidated_Data", index=False)

    print(f"Data exported to {output_path}")

# 4. Example Usage (replace paths with your actual paths)
# pdf_path = "path_to_your_pdf_file.pdf"
# output_excel_path = "path_for_output_excel_file.xlsx"
# convert_pdf_to_excel_consolidated(pdf_path, output_excel_path)
